<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%2014%20-%20Entrenamiento%20Distribuido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalación de dependencias y configuración del entorno

In [1]:
!pip install tensorflow-gpu==2.4.0

     |████████████████████████████████| 394.7MB 43kB/s 


## Paso 2: Importar las dependencias del proyecto

In [2]:
import time
import numpy as np
import tensorflow as tf
#tf.debugging.set_log_device_placement(True)

In [3]:
tf.__version__

'2.4.0'

## Paso 3: Pre procesado del dataset

### Cargar el dataset del MNIST

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Normalización de las imágenes

In [5]:
X_train = X_train / 255.
X_test = X_test / 255.

In [6]:
X_train.shape

(60000, 28, 28)

### Redimensionar el dataset

In [7]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [8]:
X_train.shape

(60000, 784)

## Paso 4: Entrenamiento distribuido

### Definir un modelo normal (no distribuido)

In [9]:
model_normal = tf.keras.models.Sequential()

In [10]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [11]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [12]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [13]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Definir una estrategia distribuida

In [14]:
# ver gpus disponibles
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [15]:
#Esto creará una instancia MirroredStrategy que usará todas las GPU que son visibles para TensorFlow y usará NCCL como comunicación entre dispositivos.
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


### Definir un modelo distribuido

In [16]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Comparar la velocidad de entrenamiento normal vs distribuida

In [17]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("El entrenamiento distribuido ha tardado: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 6s 1ms/step - loss: 0.4539 - sparse_categorical_accuracy: 0.8678
Epoch 2/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.1465 - sparse_categorical_accuracy: 0.9562
Epoch 3/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.1001 - sparse_categorical_accuracy: 0.9704
Epoch 4/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0836 - sparse_categorical_accuracy: 0.9746
Epoch 5/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0699 - sparse_categorical_accuracy: 0.9771
Epoch 6/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0598 - sparse_categorical_accuracy: 0.9814
Epoch 7/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0539 - sparse_categorical_accuracy: 0.9828
Epoch 8/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0496 - sparse_categorical_accuracy: 0.9837
Epoch 9/10
2400/2400 [==========

In [18]:
start_time = time.time()
with tf.device('/CPU:0'):
  model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("El entrenamiento normal ha tardado: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 4s 1ms/step - loss: 0.4609 - sparse_categorical_accuracy: 0.8664
Epoch 2/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.1433 - sparse_categorical_accuracy: 0.9583
Epoch 3/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.1006 - sparse_categorical_accuracy: 0.9694
Epoch 4/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0809 - sparse_categorical_accuracy: 0.9747
Epoch 5/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0717 - sparse_categorical_accuracy: 0.9774
Epoch 6/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0619 - sparse_categorical_accuracy: 0.9801
Epoch 7/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0526 - sparse_categorical_accuracy: 0.9830
Epoch 8/10
2400/2400 [==============================] - 3s 1ms/step - loss: 0.0511 - sparse_categorical_accuracy: 0.9829
Epoch 9/10
2400/2400 [==========

En este caso podemos ver que el entrenamieto distribuido demora más que el entrenamientono no distribuido esto parece contra intuitivo pero esto se debe a que es un ejemplo muy pequeño el cual no necesita entrenamiento distribuido y se pierde un poco de tiempo en los procesos por debajo para aplicar la estrategia distribuida, todo esto cambia en un problema grande en el cual realmente una estrategia distribuida será mucho mas veloz.